In [1]:
!pip install -q datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


If running this notebook in Colab, please ensure that your Hugging Face `HF_TOKEN` and your Weights & Biases `WANDB_API_KEY` are added to your Colab secrets.

Alternatively, please login to Hugging Face and Weights & Biases by running the following two cells.

In [3]:
# !huggingface-cli login

In [4]:
# !wandb login

In [5]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [6]:
from datasets import load_dataset

iris = load_dataset("scikit-learn/iris")
iris

Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species'],
        num_rows: 150
    })
})

In [7]:
features = iris['train'].features
features

{'Id': Value(dtype='int64', id=None),
 'SepalLengthCm': Value(dtype='float64', id=None),
 'SepalWidthCm': Value(dtype='float64', id=None),
 'PetalLengthCm': Value(dtype='float64', id=None),
 'PetalWidthCm': Value(dtype='float64', id=None),
 'Species': Value(dtype='string', id=None)}

In [8]:
iris.set_format("pandas")
iris_df = iris['train'][:]
iris_df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [9]:
iris_df['Species'].value_counts()

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [10]:
label2id = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
iris_df['Species'] = [label2id[species] for species in iris_df['Species']]
iris_df['Species'].value_counts()

Species
0    50
1    50
2    50
Name: count, dtype: int64

In [11]:
iris_df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [12]:
iris_df = iris_df.sample(frac=1, replace=False, random_state=42).reset_index(drop=True)
iris_df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,74,6.1,2.8,4.7,1.2,1
1,19,5.7,3.8,1.7,0.3,0
2,119,7.7,2.6,6.9,2.3,2
3,79,6.0,2.9,4.5,1.5,1
4,77,6.8,2.8,4.8,1.4,1
...,...,...,...,...,...,...
145,72,6.1,2.8,4.0,1.3,1
146,107,4.9,2.5,4.5,1.7,2
147,15,5.8,4.0,1.2,0.2,0
148,93,5.8,2.6,4.0,1.2,1


In [13]:
X = iris_df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
y = iris_df['Species'].values
X.shape, y.shape

((150, 4), (150,))

In [14]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train_full.shape, X_test.shape, y_train_full.shape, y_test.shape

((135, 4), (15, 4), (135,), (15,))

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, stratify=y_train_full, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((121, 4), (14, 4), (121,), (14,))

In [16]:
X_means, X_stds = X_train.mean(axis=0), X_train.std(axis=0)
X_means, X_stds

(array([5.82479339, 3.02809917, 3.7214876 , 1.18264463]),
 array([0.82193895, 0.43980145, 1.75476714, 0.75434583]))

In [17]:
X_train = (X_train - X_means) / X_stds
X_valid = (X_valid - X_means) / X_stds
X_test = (X_test - X_means) / X_stds
X_train.shape, X_valid.shape, X_test.shape

((121, 4), (14, 4), (15, 4))

In [18]:
from torch.utils.data import Dataset, DataLoader

class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [19]:
train_set = IrisDataset(X_train, y_train)
len(train_set)

121

In [20]:
# Sanity check:
train_set[0]

(tensor([ 1.0648, -1.2008,  1.1845,  0.8184]), tensor(2))

In [21]:
valid_set = IrisDataset(X_valid, y_valid)
test_set = IrisDataset(X_test, y_test)
len(valid_set), len(test_set)

(14, 15)

In [22]:
batch_size = 8
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
len(train_loader)

16

In [23]:
# Sanity check:
x_batch, y_batch = next(iter(train_loader))
x_batch.shape, y_batch.shape

(torch.Size([8, 4]), torch.Size([8]))

In [24]:
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
len(valid_loader), len(test_loader)

(2, 2)

In [25]:
device = torch.device("cpu")
device

device(type='cpu')

In [26]:
import torch.nn as nn
from huggingface_hub import PyTorchModelHubMixin

class MLP(nn.Module, PyTorchModelHubMixin):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 5)
        self.fc2 = nn.Linear(5, 3)

    def forward(self, x):
        act = torch.relu(self.fc1(x))
        return self.fc2(act)

In [27]:
model = MLP().to(device)
model

MLP(
  (fc1): Linear(in_features=4, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
)

In [28]:
# Sanity check:
x_batch = x_batch.to(device)
with torch.no_grad():
    logits = model(x_batch)
logits.shape

torch.Size([8, 3])

In [29]:
loss_fn = nn.CrossEntropyLoss(reduction='sum')

In [30]:
# Sanity check:
y_batch = y_batch.to(device)
with torch.no_grad():
    loss = loss_fn(logits, y_batch)
loss

tensor(9.9795)

In [31]:
from torch.optim import SGD

learning_rate = 1e-2
optimizer = SGD(model.parameters(), lr=learning_rate)
n_epochs = 100

In [32]:
def train_epoch():
    model.train()
    train_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        logits = model(x_batch)
        loss = loss_fn(logits, y_batch)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_set)
    train_loss = round(train_loss, 4)
    return train_loss

In [33]:
import evaluate

def validate_epoch():
    model.eval()
    valid_loss = 0
    metric = evaluate.load("accuracy")
    for x_batch, y_batch in valid_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        with torch.no_grad():
            logits = model(x_batch)
            loss = loss_fn(logits, y_batch)
        valid_loss += loss.item()
        preds = logits.argmax(dim=-1)
        metric.add_batch(predictions=preds, references=y_batch)
    valid_loss /= len(valid_set)
    valid_loss = round(valid_loss, 4)
    computed_metric = metric.compute()
    acc = round(computed_metric['accuracy'], 4)
    return valid_loss, acc

In [34]:
import wandb

wandb_config = {
    'architecture': "MLP",
    'dataset': "Iris",
    'batch_size': batch_size,
    'learning_rate': learning_rate,
    'n_epochs': n_epochs,
}
wandb.init(
    project="mlp-iris",
    config=wandb_config,
    notes="Logging min of `train_loss` & `valid_loss`, and max of `accuracy`."
)
wandb.define_metric("train_loss", summary="min")
wandb.define_metric("valid_loss", summary="min")
wandb.define_metric("accuracy", summary="max")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [35]:
best_epoch = 0
best_valid_loss = float('inf')
best_acc = 0
for epoch in range(n_epochs):
    train_loss = train_epoch()

    valid_loss, acc = validate_epoch()

    wandb.log({'train_loss': train_loss, 'valid_loss': valid_loss, 'accuracy': acc})

    print(f"Epoch: {epoch}, Training Loss: {train_loss}, Validation Loss: {valid_loss}, Accuracy: {acc}")

    if valid_loss < best_valid_loss:
        best_epoch = epoch
        best_valid_loss = valid_loss
        model.push_to_hub("mlp-iris", commit_message=f"epoch: {epoch}, valid_loss: {valid_loss}, accuracy: {acc}")

    if acc > best_acc:
        best_acc = acc
    print("---")
wandb.finish()
print("---")
print("Done!")
print(f"Best Epoch: {best_epoch}, Best Validation Loss: {best_valid_loss}, Best Accuracy: {best_acc}")

Epoch: 0, Training Loss: 1.1113, Validation Loss: 1.0528, Accuracy: 0.6429


model.safetensors:   0%|          | 0.00/444 [00:00<?, ?B/s]

---
Epoch: 1, Training Loss: 1.0276, Validation Loss: 0.9581, Accuracy: 0.6429


model.safetensors:   0%|          | 0.00/444 [00:00<?, ?B/s]

---
Epoch: 2, Training Loss: 0.9367, Validation Loss: 0.8608, Accuracy: 0.6429


...

---
Epoch: 98, Training Loss: 0.0652, Validation Loss: 0.0128, Accuracy: 1.0


model.safetensors:   0%|          | 0.00/444 [00:00<?, ?B/s]

---
Epoch: 99, Training Loss: 0.0654, Validation Loss: 0.0129, Accuracy: 1.0
---


accuracy,▁▁▄▄█▇▇█████████████████████████████████
train_loss,█▇▅▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▇▅▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


---
Done!
Best Epoch: 98, Best Validation Loss: 0.0128, Best Accuracy: 1.0


In [36]:
model = MLP.from_pretrained("sadhaklal/mlp-iris")
model = model.to(device)
model

model.safetensors:   0%|          | 0.00/444 [00:00<?, ?B/s]

MLP(
  (fc1): Linear(in_features=4, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
)

In [37]:
model.eval()
metric = evaluate.load("accuracy")
for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    with torch.no_grad():
        logits = model(x_batch)
    preds = logits.argmax(dim=-1)
    metric.add_batch(predictions=preds, references=y_batch)
computed_metric = metric.compute()
acc = round(computed_metric['accuracy'], 4)
print(f"Test Set Accuracy: {acc}")

Test Set Accuracy: 0.9333
